In [ ]:
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

data_path = r"C:\Users\ASTITVA RANJAN\Desktop\internship\cleaned_customer_data.csv"
data = pd.read_csv(data_path)

data.replace([np.inf, -np.inf], np.nan, inplace=True)
critical_columns = ["customer_age", "purchase_amount", "product_rating", "product_category", "loyalty_status"]
data.dropna(subset=critical_columns, inplace=True)


plt.figure(figsize=(6, 4))
sns.histplot(data["customer_age"], bins=10, kde=True)
plt.title("Customer Age Distribution")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()


plt.figure(figsize=(8, 5))
sns.barplot(data=data, x="product_category", y="purchase_amount", errorbar=None)  
plt.title("Purchase Amount by Product Category")
plt.xticks(rotation=45)
plt.show()


plt.figure(figsize=(6, 4))
loyalty_counts = data["loyalty_status"].value_counts()
loyalty_counts.plot(kind="pie", autopct="%1.1f%%")
plt.title("Loyalty Status Distribution")
plt.ylabel("") 
plt.show()

data["churn"] = np.where(data["loyalty_status"] == "Standard", 1, 0)

X = data[["customer_age", "purchase_amount", "product_rating"]]
y = data["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred, zero_division=0))  



from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


apikey = "your_actual_api_key"  
url = "your_actual_service_url"  

if not apikey or not url:
    print(" IBM Watson API is bypassed due to missing credentials.")
else:
    authenticator = IAMAuthenticator(apikey)
    nlu = NaturalLanguageUnderstandingV1(
        version='2021-08-01',
        authenticator=authenticator
    )
    nlu.set_service_url(url)

    customer_feedback = [
        "I love the new features of the product!",
        "The service was terrible; I had to wait too long.",
        "Not what I expected, but the support team helped me.",
        "Fantastic experience overall! I will recommend it to others."
    ]

    def analyze_feedback(feedback):
        overall_sentiment = {'positive': 0, 'neutral': 0, 'negative': 0}
        keywords_summary = {}

        for text in feedback:
            try:
                response = nlu.analyze(
                    text=text,
                    features={'sentiment': {}, 'keywords': {}}
                ).get_result()

                sentiment = response['sentiment']['document']['label']
                overall_sentiment[sentiment] += 1

                keywords = response.get('keywords', [])
                for keyword in keywords:
                    word = keyword.get('text')
                    keywords_summary[word] = keywords_summary.get(word, 0) + 1

            except Exception as e:
                print(f"Error analyzing text: {text}\nError: {e}")

        print("\nOverall Sentiment Summary:")
        print(overall_sentiment)
        print("\nKeywords Summary:")
        print(keywords_summary)

    analyze_feedback(customer_feedback)
